# Data Collection Notebook

# Imports

In [4]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
import pandas as pd
from IPython.display import display
import time
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm 
import usaddress  
from functools import partial
import tempfile
import shutil
import glob


notebooks_folder = os.getcwd() 


raw_root_folder = os.path.abspath(os.path.join(notebooks_folder, "..", "data", "raw", "New York City Sales Data"))
interim_root_folder = os.path.abspath( os.path.join(notebooks_folder, "..", "data", "interim", "New York City Sales Data"))
processed_root_folder = os.path.abspath( os.path.join(notebooks_folder, "..", "data", "processed", "New York City Sales Data"))
os.makedirs(raw_root_folder, exist_ok=True)
os.makedirs(interim_root_folder, exist_ok=True)  
os.makedirs(processed_root_folder, exist_ok=True)  


# NYC Rolling Sales Data Collection

## Initial Parsing of the Webpage 

Looking at the HTML layout of the website, we can see that there are `<table>` elements. These contain the rolling sales data. We can parse all the table rows `<tr>` from the webpage, and begin filtering from there.  

In [5]:
url = "https://www.nyc.gov/site/finance/property/property-annualized-sales-update.page"  # replace with your site
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

rows = soup.find_all("tr")

for r in rows:
    print(r)

<tr style="height: 36px;">
<td colspan="3" style="background-color: #00539a; color: #ffffff; height: 36px;"><strong>Neighborhood Sales Summary<br/></strong>Statistical summary of sales for all five boroughs for one-, two-, and three-family homes</td>
</tr>
<tr class="header" style="height: 36px;">
<td style="height: 36px;"><strong>Neighborhood Sales Data <br/></strong></td>
<td style="text-align: center; height: 36px;"><strong><em>Adobe PDF</em></strong></td>
<td style="text-align: center; height: 36px;"><em><strong>MS Excel</strong></em></td>
</tr>
<tr>
<td>Manhattan</td>
<td style="text-align: center;"><a href="/assets/finance/downloads/pdf/rolling_sales/neighborhood_sales/2024/2024_manhattan.pdf" rel="noopener" target="_blank">Download</a></td>
<td style="text-align: center;"><a href="/assets/finance/downloads/pdf/rolling_sales/neighborhood_sales/2024/2024_manhattan.xlsx" rel="noopener" target="_blank">Download</a></td>
</tr>
<tr>
<td>Bronx</td>
<td style="text-align: center;"><a hr

## Web Scrapping Script

From the html we parsed previously, there are a few things to note 

1) There is a title change in the `<td>` from 2016 to 2014. These say `<yyyy> New York City` as opposed to `<yyyy> New York City Sales Data`
2) From 2003 - 2017, the legacy extension for excell files `.xls` is used. This is changed to `.xlsx` from 2018 - 2024

The Data Saving Structure is as follows: 
```text
New York City Sales Data/
├── 2003/
│   ├── Manhattan.xls
│   ├── Bronx.xls
│   ├── Brooklyn.xls
│   ├── Queens.xls
│   └── Staten Island.xls
├── 2004/
│   ├── Manhattan.xls
│   ├── Bronx.xls
│   ├── Brooklyn.xls
│   ├── Queens.xls
│   └── Staten Island.xls
...
└── 2018/
    ├── Manhattan.xlsx
    ├── Bronx.xlsx
    ├── Brooklyn.xlsx
    ├── Queens.xlsx
    └── Staten Island.xlsx


In [20]:
rows = soup.find_all("tr")

current_year = None

for tr in rows:
    cells = tr.find_all("td")
    if not cells:
        continue

    text = cells[0].get_text(strip=True)

    # Due to table header change in td, we use regex to find the heading and year
    match = re.search(r"(\d{4})\s+New\s+York\s+City", text)
    if match:
        current_year = match.group(1)
        year_folder = os.path.join(raw_root_folder, current_year)
        os.makedirs(year_folder, exist_ok=True)
        print(f"\nSaving files for {current_year}")
        continue

    if current_year is None:
        continue

    borough = cells[0].get_text(strip=True)
    link_tag = cells[2].find("a") if len(cells) > 2 else None
    if not link_tag or not link_tag.has_attr("href"):
        continue

    excel_href = link_tag["href"]
    excel_url = urljoin(url, excel_href)
    

    filename = f"{borough}.xlsx" if ".xlsx" in excel_url else f"{borough}.xls"
    filepath = os.path.join(year_folder, filename)

    print(f"Downloading {borough} {current_year} data")
    with requests.get(excel_url, stream=True) as r:
        r.raise_for_status()
        with open(filepath, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

print("All files downloaded")


Saving files for 2024

Saving files for 2023

Saving files for 2022

Saving files for 2021

Saving files for 2020

Saving files for 2019

Saving files for 2018

Saving files for 2017

Saving files for 2016

Saving files for 2015

Saving files for 2014

Saving files for 2013

Saving files for 2012

Saving files for 2011

Saving files for 2010

Saving files for 2009

Saving files for 2008

Saving files for 2007

Saving files for 2006

Saving files for 2005

Saving files for 2004

Saving files for 2003
All files downloaded


# NYC Rolling Sales Data Filtering & Unique Address Collection

## Filtering Pipeline Summary

This cell performs a full cleaning, filtering, and address-preparation pipeline for NYC property sales files (reads from `raw_root_folder`, writes cleaned `.xlsx` files into `interim_root_folder`).

### Data Filtering Main steps performed (in order)

1. **File discovery & read**
   - Iterates year subfolders in `raw_root_folder`.  
   - Skips temporary files and non-Excel files (`.xls` / `.xlsx`).
   - Reads the file once with `header=None` to detect the header row. The read uses `dtype=object` so columns are not auto-coerced into numeric types.

2. **Early illegal-character cleaning**
   - Applies `clean_column_name` / `clean_illegal_chars` to object/string cells to remove control characters that will break Excel (`ASCII 0-31`, etc.).
   - This prevents `IllegalCharacterError` when saving.

3. **Header detection and realignment**
   - Normalizes header candidates and matches them against `expected_columns`.
   - Re-reads the file once using the detected header row (`header=header_row_idx`) and `dtype=object`, then normalizes column names.

4. **Type conversions**
   - Converts integer columns (`BLOCK`, `LOT`, `RESIDENTIAL UNITS`, etc.) to pandas `Int64` (nullable integer).
   - Converts numeric columns (`LAND SQUARE FEET`, `GROSS SQUARE FEET`, `SALE PRICE`) to numeric (`float`).
   - Normalizes string columns (casts to `str`, replaces `"nan"` placeholders).
   - Parses `SALE DATE` with `pd.to_datetime(..., errors="coerce")`.

5. **Residential filtering**
   - Normalizes `BUILDING CLASS CATEGORY` strings.
   - Keeps only rows that match residential keywords (`FAMILY`, `RENTAL`, `COOP`, `CONDO`, `CONDOP`, `TAX CLASS 1`).

6. **Address parsing & cleaning**
   - Uses `usaddress` to parse street components and only retains addresses that contain both a house number and a street name.
   - Builds `ADDRESS_CLEAN` from parsed components (number + street types/names).

7. **Remove non-market transactions**
   - Drops rows where `SALE PRICE` ≤ 0 (these are typically non-arms-length or administrative transfers, not market sales).

8. **Build `FULL_ADDRESS` for geocoding**
   - Maps `BOROUGH` → appropriate city label for geocoders (Manhattan → `New York`, Brooklyn → `Brooklyn`, Queens → `Queens`, Bronx → `Bronx`, Staten Island → `Staten Island`).
   - Assembles `FULL_ADDRESS` as:
     ```
     [ADDRESS_CLEAN], [CITY], NY, [ZIP CODE]
     ```
     Example: `123 MAIN ST, BROOKLYN, NY, 11215`.

9. **Column selection**
   - Keeps only the minimal columns needed for mapping/visualization:
     - `FULL_ADDRESS`  
     - `SALE PRICE`  
     - `SALE DATE`

10. **Save & convert**
    - Ensures `interim_root_folder/<year>/` exists and saves cleaned data as `.xlsx` into that folder.
    - If the original file was `.xls`, the script removes the old `.xls` after saving the cleaned `.xlsx`.

### Final output
- For each processed raw file, you get a cleaned `.xlsx` in `interim_root_folder/<year>/` containing only:
     - `FULL_ADDRESS`  
     - `SALE PRICE`  
     - `SALE DATE`

## Unique Address Collection 

After processing and cleaning all raw property sales files, we collect the **unique addresses** for geocoding and mapping purposes.

1. **Track unique addresses while processing**
   - Each cleaned `FULL_ADDRESS` from the current file is added to a Python `set()` to automatically ensure uniqueness:
     ```python
     unique_addresses.update(df["FULL ADDRESS"])
     ```

2. **Convert to DataFrame**
   - Convert the set of unique addresses into a sorted pandas DataFrame:
     ```python
     unique_df = pd.DataFrame(sorted(unique_addresses), columns=["FULL ADDRESS"])
     ```

3. **Assign round-robin groups**
   - Assign each address to a deterministic group (useful for batch processing or parallel geocoding):
     ```python
     n_groups = 6
     unique_df["GROUP"] = (unique_df.index % n_groups) + 1
     ```

4. **Add blank latitude/longitude columns for future geocoding**
   - Prepare columns to store geocoded coordinates later:
     ```python
     unique_df["LAT"] = None
     unique_df["LON"] = None
     ```

5. **Reorder columns**
   - Ensure the final column order is consistent:
     ```python
     unique_df = unique_df[["GROUP", "FULL ADDRESS", "LAT", "LON"]]
     ```

6. **Save to Excel**
   - Save the unique addresses and groups to an Excel file in the `interim_root_folder`:
     ```python
     geocode_cache_path = os.path.join(interim_root_folder, "unique_addresses.xlsx")
     unique_df.to_excel(geocode_cache_path, index=False, engine="openpyxl")
     ```

### Final Output
- `unique_addresses.xlsx` containing:
  - `GROUP` → deterministic assignment for batch processing
  - `FULL ADDRESS` → cleaned, geocodable addresses
  - `LAT` / `LON` → blank for now, to be filled later by geocoding
- Ensures **all addresses are unique** across years and files.


In [21]:
# ==============================
# config
# ==============================
expected_columns = [
    "BOROUGH", "NEIGHBORHOOD", "BUILDING CLASS CATEGORY", "TAX CLASS AT PRESENT",
    "BLOCK", "LOT", "EASE-MENT", "BUILDING CLASS AT PRESENT", "ADDRESS",
    "APARTMENT NUMBER", "ZIP CODE", "RESIDENTIAL UNITS", "COMMERCIAL UNITS",
    "TOTAL UNITS", "LAND SQUARE FEET", "GROSS SQUARE FEET", "YEAR BUILT",
    "TAX CLASS AT TIME OF SALE", "BUILDING CLASS AT TIME OF SALE",
    "SALE PRICE", "SALE DATE"
]

int_cols = ["BLOCK", "LOT", "RESIDENTIAL UNITS", "COMMERCIAL UNITS", "TOTAL UNITS", "YEAR BUILT"]
float_cols = ["LAND SQUARE FEET", "GROSS SQUARE FEET", "SALE PRICE"]
str_cols = ["BOROUGH", "NEIGHBORHOOD", "BUILDING CLASS CATEGORY", "TAX CLASS AT PRESENT",
            "EASE-MENT", "BUILDING CLASS AT PRESENT", "ADDRESS", "APARTMENT NUMBER",
            "ZIP CODE", "TAX CLASS AT TIME OF SALE", "BUILDING CLASS AT TIME OF SALE"]
datetime_cols = ["SALE DATE"]

residential_keywords = [
    "FAMILY", "RENTAL", "COOP", "CONDO", "CONDOP", "TAX CLASS 1"
]

borough_encoding__to_city_map = {"1": "New York", "2": "Bronx", "3": "Brooklyn", "4": "Queens", "5": "Staten Island"}
unique_addresses = set()

# ==============================
# REGEX CLEANING HELPER FUNCTIONS
# ==============================
def normalize(col):
    return str(col).replace('\n', ' ').replace('"', '').replace("  ", " ").strip().upper()

def clean_column_name(s):
    if isinstance(s, str):
        # Remove illegal characters (ASCII 0-31 except \t, \n, \r)
        s = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F]', '', s)
    return s

def clean_illegal_chars(val, replace_with=""):
    """Remove Excel-illegal control chars from a string.
       replace_with: '' (remove) or ' ' (replace with space)
    """
    if isinstance(val, str):
        # remove ASCII 0-31 and 127-159
        return re.sub(r'[\x00-\x1F\x7F-\x9F]+', replace_with, val).strip()
    return val

# ==============================
# PROPERTY TYPE CLEANING
# ==============================
def is_residential(category: str) -> bool:
    """Check if a category is residential."""
    if not isinstance(category, str):
        return False
    category = category.upper()
    return any(k in category for k in residential_keywords)


# ==============================
# ADDRESS CLEANING
# ==============================
def clean_address(addr: str) -> str | None:
    """Try to parse & clean address. Returns cleaned address or None if invalid."""
    if not isinstance(addr, str) or not addr.strip():
        return None

    addr = re.sub(r"\s+", " ", addr.strip().title())

    try:
        parsed, _ = usaddress.tag(addr)
        # Keep only addresses that have street + house number
        if "AddressNumber" in parsed and "StreetName" in parsed:
            # Rebuild normalized street address
            parts = [
                parsed.get("AddressNumber", ""),
                parsed.get("StreetNamePreType", ""),
                parsed.get("StreetName", ""),
                parsed.get("StreetNamePostType", "")
            ]
            clean = " ".join([p for p in parts if p]).strip()
            return clean
        else:
            return None
    except usaddress.RepeatedLabelError:
        return None
    
# ==============================
# FULL ADDRESS ASSEMBLY
# ==============================
def build_full_address(row):
    parts = [row["ADDRESS_CLEAN"].strip().title()]

    # Add borough→city
    city = borough_encoding__to_city_map[row["BOROUGH"]].strip().title()
    parts.append(row["NEIGHBORHOOD"].strip().title())
    parts.append(city)
    parts.append("NY")
    parts.append(str(row["ZIP CODE"]).strip().title())

    return ", ".join([p for p in parts if p])

# ==============================
# MAIN PROCESSING LOOP
# ==============================
for year in os.listdir(raw_root_folder):
    year_folder = os.path.join(raw_root_folder, year)
    if not os.path.isdir(year_folder):
        continue
    print(f"\n=== Entering year: {year} ===")

    for file in os.listdir(year_folder):

        # skip non-Excel files or temp excel files
        if not file.lower().endswith((".xls", ".xlsx")) or file.startswith("~"):  
            continue

        # Read in file (.xls or .xlsx)
        file_path = os.path.join(year_folder, file)
        try:
            df = pd.read_excel(file_path, header=None, engine="openpyxl", dtype=object)
        except Exception:
            df = pd.read_excel(file_path, header=None, engine="xlrd", dtype=object)

        # display(df.head(10))

        # Clean illegal characters before anything else
        for col in df.select_dtypes(include="object"):
            df[col] = df[col].apply(clean_column_name)

        # Detect header row
        normalized_expected = [normalize(c) for c in expected_columns]
        header_row_idx = None
        for i, row in df.iterrows():
            normalized_row = [normalize(c) for c in row.values]
            matches = sum(col in normalized_expected for col in normalized_row)
            if matches >= len(normalized_expected) * 0.7:
                header_row_idx = i
                break

        # read in file, now with proper header alignment
        try:
            df = pd.read_excel(file_path, header=header_row_idx, engine="openpyxl", dtype=object)
        except Exception:
            df = pd.read_excel(file_path, header=header_row_idx, engine="xlrd", dtype=object)
        df.columns = [normalize(c) for c in df.columns]
        df.reset_index(drop=True, inplace=True)

        # display(df.head(10))
        
        # Convert cols to proper data types
        for col in int_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

        for col in float_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors="coerce")

        for col in str_cols:
            if col in df.columns:
                df[col] = df[col].astype(str).replace("nan", "")

        for col in datetime_cols:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col], errors="coerce")

        # tracking how much data we are removing
        before_count = len(df)
        
        # Cleaning residential categories
        df["BUILDING CLASS CATEGORY"] = (
            df["BUILDING CLASS CATEGORY"]
            .astype(str)
            .apply(lambda x: re.sub(r"\s+", " ", x.strip()))
        )
        df = df[df["BUILDING CLASS CATEGORY"].apply(is_residential)]

        #  Cleaning address
        df["ADDRESS_CLEAN"] = df["ADDRESS"].apply(clean_address)
        df = df[df["ADDRESS_CLEAN"].notna()]

        # Removing $0 sales
        df = df[df['SALE PRICE'] > 0]

        # Removing 0 ZIP CODE
        df = df[df['ZIP CODE'] != "0"]

        # Borough → city conversion and full address building
        df["FULL ADDRESS"] = df.apply(build_full_address, axis=1)

        # Keep only relevant columns for mapping/analysis
        keep_cols = [
            "FULL ADDRESS",
            "SALE PRICE",
            "SALE DATE",
        ]
        df = df[[c for c in keep_cols if c in df.columns]]

        # Print out % of removed data
        after_count = len(df)
        filtered_out = before_count - after_count
        print(f"{file}: Kept {after_count:,} rows, filtered out {filtered_out:,} ({filtered_out / before_count:.1%})")

        # Create new file path with .xlsx extension for consistent file types
        interim_year_folder = os.path.join(interim_root_folder, year)
        os.makedirs(interim_year_folder, exist_ok=True)
        new_file_path = os.path.join(interim_year_folder, os.path.splitext(file)[0] + ".xlsx")

        # Clean all object/string columns before saving to Excel
        for col in df.select_dtypes(include="object"):
            df[col] = df[col].apply(clean_illegal_chars)

        # Tracks unique addresses seen
        # Will be used later to get lat and longs
        unique_addresses.update(df["FULL ADDRESS"])

        # Save back to the same Excel file
        df.to_excel(new_file_path, index=False, engine="openpyxl")

print("Done filtering!")


print("Creating address geocache data frame")
# Convert to DataFrame
addresses_df = pd.DataFrame(sorted(unique_addresses), columns=["FULL ADDRESS"])

# Add blank LAT/LON columns
addresses_df["LAT"] = None
addresses_df["LON"] = None

# Reorder columns
addresses_df = addresses_df[["FULL ADDRESS", "LAT", "LON"]]

# Save CSV
geocode_cache_path = os.path.join(interim_root_folder, "addresses.xlsx")
addresses_df.to_excel(geocode_cache_path, index=False, engine="openpyxl")
print(f"Saved {len(addresses_df)} unique addresses to {geocode_cache_path}")





=== Entering year: 2003 ===
Bronx.xls: Kept 6,408 rows, filtered out 4,256 (39.9%)
Brooklyn.xls: Kept 18,884 rows, filtered out 14,822 (44.0%)
Manhattan.xls: Kept 17,449 rows, filtered out 4,761 (21.4%)
Queens.xls: Kept 24,623 rows, filtered out 14,554 (37.1%)
Staten Island.xls: Kept 7,516 rows, filtered out 5,155 (40.7%)

=== Entering year: 2004 ===
Bronx.xls: Kept 7,422 rows, filtered out 4,385 (37.1%)
Brooklyn.xls: Kept 21,746 rows, filtered out 13,302 (38.0%)
Manhattan.xls: Kept 21,640 rows, filtered out 4,254 (16.4%)
Queens.xls: Kept 27,918 rows, filtered out 13,281 (32.2%)
Staten Island.xls: Kept 8,287 rows, filtered out 4,356 (34.5%)

=== Entering year: 2005 ===
Bronx.xls: Kept 7,705 rows, filtered out 4,100 (34.7%)
Brooklyn.xls: Kept 22,059 rows, filtered out 12,184 (35.6%)
Manhattan.xls: Kept 21,564 rows, filtered out 4,824 (18.3%)
Queens.xls: Kept 28,201 rows, filtered out 11,736 (29.4%)
Staten Island.xls: Kept 7,580 rows, filtered out 3,690 (32.7%)

=== Entering year: 2006 

## Advanced Address Cleaning & Normalization for Geocoding

This step performs a **comprehensive cleaning, normalization, and deduplication** of New York City property addresses. The primary purpose is to **reduce the number of geocoding lookups** by condensing multiple variations of the same address into a single normalized form. Once we have the normalized addresses, we can use them as a **key to append latitude and longitude** to our NYC sales data.

### Key Features of the Workflow

1. **Initial Cleaning**
   - Removes illegal/control characters that can break Excel (`ASCII 0-31`, `\x7F`).
   - Strips stray leading punctuation or brackets (e.g., `(56 Street, Bensonhurst, Brooklyn, NY, 11010)` → `56 Street, Bensonhurst,Brooklyn, NY, 11010`).
   - Removes excessive leading zeros in street numbers (e.g., `0000 100th Street` → `100th Street`).
   - Normalizes spacing and capitalization.

2. **Parsing Components**
   - Splits `FULL ADDRESS` into:
     - `STREET`
     - `CITY`
     - `STATE`
     - `ZIP`
   - This allows fine-grained normalization of street names.

3. **Street Name Normalization**
   - Maps abbreviations to full street type names:
     - `AVE`, `AV`, `AVE.` → `Avenue`
     - `ST`, `ST.` → `Street`
     - `RD` → `Road`
     - `PL` → `Place`
     - etc.
   - Normalizes street ordinals:
     - Numeric-only street numbers like `107` → `107th`
     - Correctly handles `1 → 1st`, `2 → 2nd`, `3 → 3rd`, and special cases `11-13 → th`.

4. **Normalized Full Address**
   - Reconstructs `FULL ADDRESS NORM` as:
     ```
     [STREET_NORM], [CITY], [STATE], [ZIP]
     ```
   - Ensures all variations of the same address map to the same normalized form, e.g.:
     ```
     1 Ascan Ave, Forest Hills, Queens, NY, 11375
     1 Ascan Avenue, Forest Hills, Queens, NY, 11375
     → 1 Ascan Avenue, Forest Hills, Queens, NY, 11375
     ```

5. **Detecting Merged Addresses**
   - Groups addresses by `FULL ADDRESS NORM`.
   - Prints only those groups where **two or more original addresses were mapped to the same normalized address**, allowing us to verify the deduplication process.

6. **Deduplication**
   - Keeps only **unique normalized addresses**.
   - Adds a `GROUP` assignment for round-robin processing (useful for batch geocoding to respect API rate limits).
   - Final output contains only the necessary columns:
     - `GROUP`
     - `FULL ADDRESS` (original, unmodified)
     - `FULL ADDRESS NORM`
     - `LAT`
     - `LON`


In [24]:
# =====================================================
# CONFIG
# =====================================================
street_map = {
    r"\bAVE\b\.?": "Avenue",
    r"\bAV\b\.?": "Avenue",
    r"\bST\b\.?": "Street",
    r"\bRD\b\.?": "Road",
    r"\bPL\b\.?": "Place",
    r"\bTER\b\.?": "Terrace",
    r"\bBLVD\b\.?": "Boulevard",
    r"\bLN\b\.?": "Lane",
    r"\bDR\b\.?": "Drive",
    r"\bCT\b\.?": "Court",
    r"\bHWY\b\.?": "Highway",
    r"\bPKWY\b\.?": "Parkway",
    r"\bSQ\b\.?": "Square",
    r"\bCTR\b\.?": "Center",
}
n_groups = 6

# =====================================================
# HELPER FUNCTIONS
# =====================================================
def clean_illegal_chars(val):
    """Remove illegal Excel characters from string cells."""
    if isinstance(val, str):
        return re.sub(r'[\x00-\x1F\x7F]', '', val)
    return val

def clean_address_text(addr: str) -> str:
    """
    Clean up weird formatting in addresses:
    - Remove stray leading punctuation/brackets.
    - Remove excessive leading zeros in street numbers (e.g., 0000 100TH -> 100TH).
    - Normalize spaces and capitalization.
    """
    if not isinstance(addr, str):
        return addr

    # Remove illegal and control characters first
    addr = clean_illegal_chars(addr)

    # Strip leading punctuation or symbols (like (, ", ', etc.)
    addr = re.sub(r'^[^\w\d]+', '', addr)

    # Fix excessive leading zeros at the beginning of street numbers
    # e.g., "0000 100TH STREET" -> "100TH STREET"
    addr = re.sub(r'^\s*0+\s*(?=\d)', '', addr)

    # Remove double spaces
    addr = re.sub(r'\s{2,}', ' ', addr)

    # Strip trailing/leading whitespace
    addr = addr.strip()

    # Title formatting for captialization
    addr = addr.title()

    return addr

def normalize_street_ordinal(street_name):
    """Normalize street ordinals like 107 → 107TH, 1 → 1ST, 2 → 2ND, etc."""
    tokens = street_name.split()
    new_tokens = []
    for t in tokens:
        if re.fullmatch(r"\d+", t):  # purely numeric
            n = int(t)
            if 10 <= n % 100 <= 20:
                suffix = "th"
            else:
                suffix = {1: "st", 2: "nd", 3: "rd"}.get(n % 10, "th")
            new_tokens.append(f"{t}{suffix}")
        else:
            new_tokens.append(t)
    return " ".join(new_tokens)

def normalize_street_name(street):
    """Normalize capitalization, abbreviations, and ordinals in street names."""
    s = str(street).strip().title()
    for pattern, repl in street_map.items():
        s = re.sub(pattern, repl, s, flags=re.IGNORECASE)
    s = re.sub(r"\s+", " ", s)
    s = normalize_street_ordinal(s)
    return s.strip()

def parse_full_address(addr):
    """Parse 'FULL ADDRESS' into [street, neighborhood, city, state, zip]."""
    parts = [p.strip() for p in str(addr).split(",")]
    if len(parts) != 5:
        return [None, None, None, None]
    street, neighborhood, city, state, zip_code = parts[0], parts[1], parts[2], parts[3], parts[4]
    return [street, neighborhood, city, state, zip_code]

# =====================================================
# MAIN PROCESSING
# =====================================================
address_cache_path = os.path.join(interim_root_folder, "addresses.xlsx")
address_cache_path_remapped = os.path.join(interim_root_folder, "addresses_condensed.xlsx")

address_df = pd.read_excel(address_cache_path, engine="openpyxl")

# Clean Address 
address_df["CLEANED FULL ADDRESS"] = address_df["FULL ADDRESS"].apply(clean_address_text)

# Split into components
address_df[["STREET", "NEIGHBORHOOD", "CITY", "STATE", "ZIP"]] = address_df["CLEANED FULL ADDRESS"].apply(
    lambda x: pd.Series(parse_full_address(x))
)
address_df["STATE"] = address_df["STATE"].str.upper()

# Normalize street names
address_df["STREET_NORM"] = address_df["STREET"].apply(normalize_street_name)

# Build normalized full address
address_df["FULL ADDRESS NORM"] = (
    address_df["STREET_NORM"].astype(str)
    + ", "
    + address_df["NEIGHBORHOOD"].astype(str)
    + ", "
    + address_df["CITY"].astype(str)
    + ", "
    + address_df["STATE"].astype(str)
    + ", "
    + address_df["ZIP"].astype(str)
)

# =====================================================
# SHOW ONLY MERGED ADDRESSES (duplicates after cleaning)
# =====================================================
grouped = (
    address_df.groupby("FULL ADDRESS NORM")["FULL ADDRESS"]
    .apply(list)
    .reset_index()
)

merged = grouped[grouped["FULL ADDRESS"].apply(lambda lst: len(lst) > 1)]


print("=== ADDRESSES THAT GOT MERGED TO THE SAME NORMALIZED FORM ===")
for _, row in merged.iterrows():
    print(f"\n→ Normalized: {row['FULL ADDRESS NORM']}")
    for orig in row["FULL ADDRESS"]:
        print(f"   - {orig}")


# =====================================================
# KEEP ONLY UNIQUE NORMALIZED ADDRESSES
# =====================================================
unique_address_df = address_df.drop_duplicates(subset=["FULL ADDRESS NORM"]).reset_index(drop=True)

# Assign groups round-robin
unique_address_df["GROUP"] = (unique_address_df.index % n_groups) + 1

# Keeping only neccessary columns
unique_address_df = unique_address_df[["GROUP", "FULL ADDRESS", "FULL ADDRESS NORM", "LAT", "LON"]]

# Printing out how much data we re-mapped 
before_count = len(address_df)
after_count = len(unique_address_df)
filtered_out = before_count - after_count
print(f"Kept {after_count:,} unique and normalized addresses, condensed down {filtered_out:,} addresses ({filtered_out / before_count:.1%})")

# Save to Excel (includes both columns for traceability)
unique_address_df.to_excel(address_cache_path_remapped, index=False, engine="openpyxl")

print(f"\n Saved cleaned, deduplicated addresses to: {address_cache_path_remapped}")
display(unique_address_df.head(10))


=== ADDRESSES THAT GOT MERGED TO THE SAME NORMALIZED FORM ===

→ Normalized: 1-3 Minetta Street, Greenwich Village-Central, New York, NY, 10012
   - 1-3 Minetta St, Greenwich Village-Central, New York, NY, 10012
   - 1-3 Minetta Street, Greenwich Village-Central, New York, NY, 10012

→ Normalized: 10-03 Cross Bay Boulevard, Jamaica Bay, Queens, NY, 11693
   - 10-03 Cross Bay Blvd., Jamaica Bay, Queens, NY, 11693
   - 10-03 Cross Bay Boulevard, Jamaica Bay, Queens, NY, 11693

→ Normalized: 10-06 Cross Bay Boulevard, Broad Channel, Queens, NY, 11693
   - 10-06 Cross Bay Blvd., Broad Channel, Queens, NY, 11693
   - 10-06 Cross Bay Boulevard, Broad Channel, Queens, NY, 11693

→ Normalized: 10-07 124Th Street, College Point, Queens, NY, 11356
   - 10-07 124Th St, College Point, Queens, NY, 11356
   - 10-07 124Th Street, College Point, Queens, NY, 11356

→ Normalized: 10-09 157Th Street, Beechhurst, Queens, NY, 11357
   - 10-09 157Th St, Beechhurst, Queens, NY, 11357
   - 10-09 157Th Street,

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Kept 514,288 unique and normalized addresses, condensed down 25,632 addresses (4.7%)

 Saved cleaned, deduplicated addresses to: C:\Users\Wonbin\PycharmProjects\NYC-Affordability-Transit-Access\data\interim\New York City Sales Data\addresses_condensed.xlsx


,GROUP,FULL ADDRESS,FULL ADDRESS NORM,LAT,LON
0,1,"(245 Shares, Elmhurst, Queens, NY, 11373","245th Shares, Elmhurst, Queens, NY, 11373",NaN,NaN
1,2,"0 232 Street, Williamsbridge, Bronx, NY, 10466","232nd Street, Williamsbridge, Bronx, NY, 10466",NaN,NaN
2,3,"0 Glennon Place, Throgs Neck, Bronx, NY, 10465","0th Glennon Place, Throgs Neck, Bronx, NY, 10465",NaN,NaN
3,4,"0 Great Kills Road, Great Kills, Staten Island...","0th Great Kills Road, Great Kills, Staten Isla...",NaN,NaN
4,5,"00 136Th Ave, Rosedale, Queens, NY, 11422","136Th Avenue, Rosedale, Queens, NY, 11422",NaN,NaN
5,6,"00 145Th Road, Springfield Gardens, Queens, NY...","145Th Road, Springfield Gardens, Queens, NY, 1...",NaN,NaN
6,1,"000 112Th Street, Forest Hills, Queens, NY, 11375","112Th Street, Forest Hills, Queens, NY, 11375",NaN,NaN
7,2,"01 17Th Street, Flatiron, New York, NY, 10003","1st 17Th Street, Flatiron, New York, NY, 10003",NaN,NaN
8,3,"04-74 48Th Ave, Long Island City, Queens, NY, ...","4-74 48Th Avenue, Long Island City, Queens, NY...",NaN,NaN
9,4,"05 16 St., Gramercy, New York, NY, 10003","5th 16th Street, Gramercy, New York, NY, 10003",NaN,NaN


## Geocoding Pipeline Overview

This cell performs **geocoding** of NYC addresses in a safe, restartable, and rate-limited way.  
It divides the work into the following main steps:

### Config
The `config` dictionary defines key parameters:
- **`input_file`** – The master Excel file containing all addresses.  
- **`group_number`** – The specific group of addresses this run will process.  
- **`user_agent`** – Custom identifier for the Nominatim API (required for fair use).  
- **`min_delay_seconds`** – Minimum delay between API requests to avoid rate limits.  
- **`flush_every`** – How often results are written to disk (in rows).  
- **`max_retries`** – Maximum number of retry attempts per address.

### Helper Functions
1. **`safe_write_excel()`**  
   Writes output safely using a temporary file, preventing corruption if the process is interrupted.  

2. **`geocode_address()`**  
   Calls the Nominatim API with NYC-specific parameters (restricts to U.S. results and English language).  

3. **`geocode_with_retries()`**  
   Retries failed geocoding requests with exponential backoff (`2, 4, 8... seconds`), up to a maximum retry count.

### Main Script Logic

1. **Load Data**
   - Reads the input Excel file and filters only the addresses for the current group.

2. **Initialize Geocoder**
   - Creates a `Nominatim` geolocator with a `RateLimiter` to ensure polite API usage.

3. **Load Cache**
   - If a per-group cache file already exists (`geocode_group_X.xlsx`), it loads it.
   - This allows the script to resume progress without re-requesting already geocoded addresses.

4. **Build Lookup Set**
   - A set of all normalized addresses (`FULL ADDRESS NORM`) already geocoded is created.
   - If an address is in this cache, it is skipped automatically.

5. **Iterate and Geocode**
   - For each new (uncached) address:
     - Calls `geocode_with_retries()`.
     - On success, records latitude and longitude.
     - On failure, records `None` for both coordinates.

6. **Flush Periodically**
   - Every `flush_every` rows, the script writes results safely to disk using `safe_write_excel()`.
   - This ensures progress is not lost if the kernel stops mid-run.

7. **Final Flush**
   - After all rows are processed, any remaining results are written to the cache file.

### Output
Each run creates or updates:

`interim/geocode_group_<group_number>.xlsx`

This file stores all geocoded results for that group and allows future runs to skip completed addresses.

- Final output contains only the necessary columns:
     - `GROUP`
     - `FULL ADDRESS` (original, unmodified)
     - `FULL ADDRESS NORM`
     - `LAT` 
     - `LON`


In [23]:
# =====================================================
# config
# =====================================================
config = {
    "input_file": "addresses_condensed.xlsx",
    "group_number": 1,  # <-- each team member sets their group
    "user_agent": "nyc_affordability_transit_access",
    "min_delay_seconds": 1.0,
    "flush_every": 500,  # how many rows before writing to disk
    "max_retries": 3,
}

# =====================================================
# HELPER FUNCTIONS
# =====================================================

def safe_write_excel(df, final_path):
    """Safely write Excel file without risk of corruption on interruption."""
    tmp_dir = tempfile.gettempdir()
    tmp_path = os.path.join(tmp_dir, f"tmp_{os.path.basename(final_path)}")
    try:
        # Write to a temporary file first
        df.to_excel(tmp_path, index=False, engine="openpyxl")
        # Atomically replace the old file
        shutil.move(tmp_path, final_path)
    except Exception as e:
        print(f"Warning: Failed to safely write {final_path}: {e}")
        if os.path.exists(tmp_path):
            os.remove(tmp_path)

def geocode_address(geolocator, address_json):
    """Call Nominatim geocode with NYC-focused parameters."""
    return geolocator.geocode(
        address_json,
        country_codes="us",         # restrict to USA
        addressdetails=True,        # return structured address info
        language="en",              # English results
    )

def geocode_with_retries(geocode_func, address, max_retries=3):
    """Take a single comma-separated address string, and retry if failed."""
    parts = [p.strip() for p in address.split(",")]
    address = f"{parts[0]}, {parts[3]}, {parts[4]}"
    attempt = 0
    while attempt <= max_retries:
        try:
            return geocode_func(address)
        except Exception as e:
            attempt += 1
            wait = 2 ** attempt
            print(f"Warn: geocode error ({attempt}/{max_retries}) for '{address}': {e}. Backing off {wait}s.")
            time.sleep(wait)
    return None

# =====================================================
# MAIN SCRIPT
# =====================================================
input_path = os.path.join(interim_root_folder, config["input_file"])
df = pd.read_excel(input_path, engine="openpyxl")

# Filter to only the given group
to_geocode_df = df[(df["GROUP"] == config["group_number"])].reset_index(drop=True)

print(f"Group {config['group_number']} has {len(to_geocode_df)} addresses to geocode.")

# Initialize geolocator with NYC-specific parameters
geolocator = Nominatim(user_agent=config["user_agent"], timeout=10)

# Use a partial so each call applies NYC bounding box parameters
geocode = RateLimiter(
    partial(geocode_address, geolocator),
    min_delay_seconds=config["min_delay_seconds"],
    error_wait_seconds=10
)

# Prepare per-group output file
group_cache_file = os.path.join(
    interim_root_folder, f"geocode_group_{config['group_number']}.xlsx"
)

# Load existing cache if available
if os.path.exists(group_cache_file):
    cache_df = pd.read_excel(group_cache_file, engine="openpyxl")
else:
    cache_df = pd.DataFrame(columns=["GROUP","FULL ADDRESS","FULL ADDRESS NORM","LAT","LON"])

# Build a set of already geocoded addresses (FULL ADDRESS NORM) for quick lookup
cache_key = set(cache_df["FULL ADDRESS NORM"].astype(str).tolist())

new_rows = []

for idx, row in tqdm(to_geocode_df.iterrows(), total=len(to_geocode_df), desc="Geocoding"):
    norm_addr = str(row["FULL ADDRESS NORM"])
    
    if norm_addr in cache_key:
        continue  # already geocoded
    
    try:
        loc = geocode_with_retries(lambda a: geocode(a), norm_addr, max_retries=config["max_retries"])
        if loc:
            lat, lon = loc.latitude, loc.longitude
        else: 
            lat, lon = None, None
    except Exception as e:
        print(f"Error geocoding '{norm_addr}': {e}")
        lat, lon = None, None

    new_rows.append({
        "GROUP": row["GROUP"],
        "FULL ADDRESS": row["FULL ADDRESS"],
        "FULL ADDRESS NORM": row["FULL ADDRESS NORM"],
        "LAT": lat,
        "LON": lon,
    })

    # Flush periodically
    if len(new_rows) >= config["flush_every"]:
        tmp_df = pd.DataFrame(new_rows)
        cache_df = pd.concat([cache_df, tmp_df], ignore_index=True)
        safe_write_excel(cache_df, group_cache_file)
        print(f"Flushed {len(new_rows)} rows to {group_cache_file} (total cached: {len(cache_df)})")
        new_rows = []

# Final flush
if new_rows:
    tmp_df = pd.DataFrame(new_rows)
    cache_df = pd.concat([cache_df, tmp_df], ignore_index=True)
    safe_write_excel(cache_df, group_cache_file)
    print(f"Final flush: {len(new_rows)} rows written to {group_cache_file} (total cached: {len(cache_df)})")

print(f"\nGeocoding complete for group {config['group_number']}. Saved results to: {group_cache_file}")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Wonbin\\PycharmProjects\\NYC-Affordability-Transit-Access\\data\\interim\\New York City Sales Data\\addresses_condensed.xlsx'

## Combining Geocode Files back into original data

In [25]:
# Base folder path
interim_root_folder = os.path.abspath(
    os.path.join(notebooks_folder, "..", "data", "interim", "New York City Sales Data")
)

# Base addresses file
base_path = os.path.join(interim_root_folder, "addresses_condensed.xlsx")
base = pd.read_excel(base_path)[["GROUP", "FULL ADDRESS", "LAT", "LON"]]

# Find all geocode group files
group_files = sorted(glob.glob(os.path.join(interim_root_folder, "geocode_group_*.xlsx")))

# Merge each group file
for file in group_files:
    print(f"Merging {os.path.basename(file)}...")
    group_df = pd.read_excel(file)
    
    # Merge LAT/LON from each geocode file
    base = base.merge(
        group_df[["GROUP", "FULL ADDRESS", "LAT", "LON"]],
        on=["GROUP", "FULL ADDRESS"],
        how="left",
        suffixes=("", "_y")
    )
    
    # Fill missing LAT/LON values
    base["LAT"] = base["LAT"].fillna(base["LAT_y"])
    base["LON"] = base["LON"].fillna(base["LON_y"])
    
    # Drop temporary columns
    base.drop(columns=["LAT_y", "LON_y"], inplace=True, errors="ignore")

# Drop rows with missing LAT or LON
base = base.dropna(subset=["LAT", "LON"])

# Keep only final columns
final = base[["FULL ADDRESS", "LAT", "LON"]]

# Save to Excel
output_path = os.path.join(interim_root_folder, "addresses_with_latlon.xlsx")
final.to_excel(output_path, index=False)

print(f"✅ Done! Saved merged file (non-empty LAT/LON only) to:\n{output_path}")
print(f"📦 Final row count: {len(final)}")


Merging geocode_group_1.xlsx...
Merging geocode_group_2.xlsx...
Merging geocode_group_3.xlsx...
Merging geocode_group_4.xlsx...
Merging geocode_group_5.xlsx...
Merging geocode_group_6.xlsx...
✅ Done! Saved merged file (non-empty LAT/LON only) to:
C:\Users\Wonbin\PycharmProjects\NYC-Affordability-Transit-Access\data\interim\New York City Sales Data\addresses_with_latlon.xlsx
📦 Final row count: 266648


In [26]:
# Path setup
latlon_path = os.path.join(interim_root_folder, "addresses_with_latlon.xlsx")
latlon_df = pd.read_excel(latlon_path)[["FULL ADDRESS", "LAT", "LON"]]

print(f"Loaded {len(latlon_df)} address records with LAT/LON data.")

for year in os.listdir(interim_root_folder):
    year_folder = os.path.join(interim_root_folder, year)
    if not os.path.isdir(year_folder):
        continue

    print(f"\n=== Entering year: {year} ===")

    for file in os.listdir(year_folder):
        # Skip non-Excel or temp Excel files
        if not file.lower().endswith((".xls", ".xlsx")) or file.startswith("~"):
            continue

        file_path = os.path.join(year_folder, file)

        print(f"Processing {file}...")
        df = pd.read_excel(file_path)

        if "FULL ADDRESS" not in df.columns:
            print(f"Skipping {file} (no 'FULL ADDRESS' column found)")
            continue

        # Merge LAT/LON from master list
        merged = df.merge(latlon_df, on="FULL ADDRESS", how="left")

        # Drop rows where no coordinates were found
        merged = merged.dropna(subset=["LAT", "LON"])

        # Overwrite same file
        merged.to_excel(file_path, index=False)
        print(f"Overwrote {file} with LAT/LON data ({len(merged)} rows kept).")

print("\nAll files updated in place with LAT/LON successfully!")

Loaded 266648 address records with LAT/LON data.

=== Entering year: 2003 ===
Processing Bronx.xlsx...
Overwrote Bronx.xlsx with LAT/LON data (1070 rows kept).
Processing Brooklyn.xlsx...
Overwrote Brooklyn.xlsx with LAT/LON data (4327 rows kept).
Processing Manhattan.xlsx...
Overwrote Manhattan.xlsx with LAT/LON data (3592 rows kept).
Processing Queens.xlsx...
Overwrote Queens.xlsx with LAT/LON data (6312 rows kept).
Processing Staten Island.xlsx...
Overwrote Staten Island.xlsx with LAT/LON data (3069 rows kept).

=== Entering year: 2004 ===
Processing Bronx.xlsx...
Overwrote Bronx.xlsx with LAT/LON data (1210 rows kept).
Processing Brooklyn.xlsx...
Overwrote Brooklyn.xlsx with LAT/LON data (4976 rows kept).
Processing Manhattan.xlsx...
Overwrote Manhattan.xlsx with LAT/LON data (4556 rows kept).
Processing Queens.xlsx...
Overwrote Queens.xlsx with LAT/LON data (16388 rows kept).
Processing Staten Island.xlsx...
Overwrote Staten Island.xlsx with LAT/LON data (3351 rows kept).

=== Ent

------

### Download NYC Borough GeoJSON

In [6]:
import requests
import json

# Download NYC borough boundaries # Clipped to Shoreline 
borough_url = "https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/NYC_Borough_Boundary/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=geojson"

response = requests.get(borough_url)
borough_geojson = response.json()

borough_path = os.path.join(raw_root_folder, "nyc_boroughs.geojson")
with open(borough_path, 'w') as f:
    json.dump(borough_geojson, f)

print(f"Downloaded: {len(borough_geojson['features'])} boroughs")

Downloaded: 5 boroughs


### Create Subway Stations GeoJSON

In [7]:
import geopandas as gpd
import pandas as pd
import requests
import zipfile
import io
import os

# Setup paths
subway_raw_folder = os.path.join(raw_root_folder, "subway_stations")
subway_processed_folder = os.path.join(processed_root_folder, "subway_stations")
os.makedirs(subway_raw_folder, exist_ok=True)
os.makedirs(subway_processed_folder, exist_ok=True)

# Download MTA GTFS data
gtfs_url = "http://web.mta.info/developers/data/nyct/subway/google_transit.zip"
response = requests.get(gtfs_url)

# Extract stops.txt from zip
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open('stops.txt') as f:
        stops_df = pd.read_csv(f)

# Save raw data
raw_stops_path = os.path.join(subway_raw_folder, "stops.csv")
stops_df.to_csv(raw_stops_path, index=False)

print(f"Downloaded {len(stops_df)} stop records")
stops_df.head()

from shapely.geometry import Point

nybb_url = "https://s-media.nyc.gov/agencies/dcp/assets/files/zip/data-tools/bytes/borough-boundaries/nybb_25c.zip"
nybb_zip_path = os.path.join(raw_root_folder, "nybb_25c.zip")
nybb_extract_dir = os.path.join(raw_root_folder, "nybb_25c")

# Extract
with zipfile.ZipFile(nybb_zip_path, "r") as z:
    z.extractall(nybb_extract_dir)

# The zip contains a nested folder also named nybb_25c
inner_dir = os.path.join(nybb_extract_dir, "nybb_25c")

print("Inner dir:", inner_dir)
print("Files:", os.listdir(inner_dir))

shp_path = os.path.join(inner_dir, "nybb.shp")

borough_gdf = gpd.read_file(shp_path)
print(borough_gdf.crs)
print(borough_gdf.head())

# Paths
nybb_outer = os.path.join(raw_root_folder, "nybb_25c")
nybb_inner = os.path.join(nybb_outer, "nybb_25c")
nybb_shp = os.path.join(nybb_inner, "nybb.shp")

# Load borough polygons
borough_gdf = gpd.read_file(nybb_shp)
borough_gdf = borough_gdf.to_crs(epsg=4326)
borough_gdf = borough_gdf[["BoroName", "geometry"]]

# Load subway parent stations
subway_raw_folder = os.path.join(raw_root_folder, "subway_stations")
stops_path = os.path.join(subway_raw_folder, "stops.csv")
stops_df = pd.read_csv(stops_path)

stations = stops_df[stops_df["location_type"] == 1].copy()
stations = stations[["stop_id", "stop_name", "stop_lat", "stop_lon"]]

stations_gdf = gpd.GeoDataFrame(
    stations,
    geometry=gpd.points_from_xy(stations["stop_lon"], stations["stop_lat"]),
    crs="EPSG:4326"
)

# Spatial join
joined = gpd.sjoin(
    stations_gdf,
    borough_gdf,
    how="left",
    predicate="within"
)

stations["borough"] = joined["BoroName"].fillna("Unknown")

print("\nStations by borough:")
print(stations["borough"].value_counts(dropna=False))

# Save as GeoJSON
subway_geojson = {
    "type": "FeatureCollection",
    "features": []
}

for _, row in stations.iterrows():
    subway_geojson["features"].append({
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["stop_lon"], row["stop_lat"]]
        },
        "properties": {
            "name": row["stop_name"],
            "stop_id": row["stop_id"],
            "borough": row["borough"]
        }
    })

output_path = os.path.join(processed_root_folder, "subway_stations.geojson")

import json
with open(output_path, "w") as f:
    json.dump(subway_geojson, f)

print("\nSaved GeoJSON:", output_path)

Downloaded 1488 stop records
Inner dir: C:\Users\Wonbin\PycharmProjects\NYC-Affordability-Transit-Access\data\raw\New York City Sales Data\nybb_25c\nybb_25c
Files: ['nybb.dbf', 'nybb.prj', 'nybb.shp', 'nybb.shp.xml', 'nybb.shx']
EPSG:2263
   BoroCode       BoroName     Shape_Leng    Shape_Area  \
0         5  Staten Island  325912.288988  1.623618e+09   
1         2          Bronx  463147.071960  1.187199e+09   
2         3       Brooklyn  726953.045551  1.934463e+09   
3         4         Queens  887905.078760  3.041419e+09   
4         1      Manhattan  359193.930112  6.366278e+08   

                                            geometry  
0  MULTIPOLYGON (((970217.022 145643.332, 970227....  
1  MULTIPOLYGON (((1012821.806 229228.265, 101278...  
2  MULTIPOLYGON (((1022227.32 152028.146, 1022078...  
3  MULTIPOLYGON (((1032452.015 154469.237, 103245...  
4  MULTIPOLYGON (((981219.056 188655.316, 980940....  

Stations by borough:
borough
Brooklyn         169
Manhattan        153
Quee

### Create HTML Visualization

In [10]:
html_content = '''<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Interactive visualization of NYC housing data and subway access</title>
    <script src="https://d3js.org/d3.v7.min.js"></script>
    <style>
        body {
            margin: 0;
            font-family: Arial, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        }
        #map {
            width: 100vw;
            height: 100vh;
            background: linear-gradient(to bottom, #a8dadc 0%, #f1faee 100%);
        }
        .borough {
            stroke: #2c3e50;
            stroke-width: 2;
            transition: all 0.3s ease;
        }
        .borough:hover {
            stroke-width: 3;
            filter: brightness(0.95);
        }
        .borough-1 { fill: #ffe6e6; }
        .borough-2 { fill: #fff4e6; }
        .borough-3 { fill: #e6f3ff; }
        .borough-4 { fill: #e6ffe6; }
        .borough-5 { fill: #f3e6ff; }
        .borough-label {
            font-size: 16px;
            font-weight: bold;
            fill: #2c3e50;
            fill-opacity: 0.5;
            text-anchor: middle;
            pointer-events: none;
            letter-spacing: 2px;
            text-transform: uppercase;
        }
        .subway-station {
            stroke: #fff;
            stroke-width: 1;
            cursor: pointer;
            filter: drop-shadow(0px 1px 2px rgba(0,0,0,0.3));
            transition: all 0.2s ease;
        }
        .tooltip {
            position: absolute;
            background: white;
            padding: 10px;
            border: 1px solid #ccc;
            border-radius: 4px;
            pointer-events: none;
            display: none;
            font-size: 13px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.2);
            max-width: 250px;
        }
        .controls {
            position: absolute;
            top: 20px;
            right: 20px;
            background: white;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.2);
            width: 240px;
            max-height: 90vh;
            overflow-y: auto;
        }
        .zoom-btn {
            display: inline-block;
            width: 50px;
            height: 50px;
            margin: 5px;
            font-size: 18px;
            cursor: pointer;
            border: 1px solid #ccc;
            background: white;
            border-radius: 4px;
        }
        .zoom-btn:hover {
            background: #f0f0f0;
        }
        .borough-filter {
            margin-top: 15px;
            padding-top: 15px;
            border-top: 1px solid #ddd;
        }
        .borough-filter label {
            display: block;
            margin: 10px 0;
            cursor: pointer;
            font-size: 14px;
        }
        .borough-filter input {
            margin-right: 8px;
            cursor: pointer;
        }
        .color-box {
            display: inline-block;
            width: 14px;
            height: 14px;
            margin-right: 5px;
            border-radius: 2px;
        }
        .title {
            position: absolute;
            top: 20px;
            left: 20px;
            background: white;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.2);
            max-width: 450px;
        }
        .title h1 {
            margin: 0;
            font-size: 22px;
            color: #333;
            line-height: 1.3;
        }
        .title p {
            margin: 8px 0 0 0;
            font-size: 14px;
            color: #666;
        }
        .stats {
            position: absolute;
            bottom: 20px;
            left: 20px;
            background: white;
            padding: 15px;
            border-radius: 8px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.2);
            min-width: 220px;
        }
        .stats h3 {
            margin: 0 0 10px 0;
            font-size: 16px;
        }
        .stat-item {
            margin: 8px 0;
            font-size: 14px;
            display: flex;
            justify-content: space-between;
        }
        .stat-label {
            color: #666;
        }
        .stat-value {
            font-weight: bold;
            color: #333;
        }
        h3 {
            margin: 0 0 12px 0;
            font-size: 16px;
        }
        .search-box {
            width: 100%;
            padding: 10px;
            margin: 10px 0;
            border: 1px solid #ccc;
            border-radius: 4px;
            font-size: 14px;
            box-sizing: border-box;
        }
        .action-buttons {
            margin-top: 15px;
            padding-top: 15px;
            border-top: 1px solid #ddd;
            display: flex;
            flex-direction: column;
        }
        .action-btn {
            width: 100%;
            padding: 12px 15px;
            margin: 5px 0;
            border: 1px solid #ccc;
            background: white;
            border-radius: 4px;
            cursor: pointer;
            font-size: 14px;
            box-sizing: border-box;
            display: block;
            text-align: center;
            height: 42px;
            line-height: 1.2;
        }
        .action-btn:hover {
            background: #f0f0f0;
        }
        .legend {
            position: absolute;
            bottom: 20px;
            right: 20px;
            background: white;
            padding: 15px;
            border-radius: 8px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.2);
            min-width: 250px;
        }
        .legend h3 {
            margin: 0 0 10px 0;
            font-size: 16px;
        }
        .legend-item {
            margin: 5px 0;
            font-size: 16px;
        }
        .highlight {
            stroke: yellow;
            stroke-width: 3;
        }
    </style>
</head>
<body>
    <div class="title">
        <h1>Interactive visualization of NYC housing data and subway access</h1>
        <p>Visualizing property sales and subway infrastructure to understand how transit proximity relates to neighborhood affordability</p>
    </div>
    
    <div id="map"></div>
    <div class="tooltip"></div>
    
    <div class="stats">
        <h3>Statistics</h3>
        <div class="stat-item">
            <span class="stat-label">Total Stations:</span>
            <span class="stat-value" id="total-stations">0</span>
        </div>
        <div class="stat-item">
            <span class="stat-label">Visible Stations:</span>
            <span class="stat-value" id="visible-stations">0</span>
        </div>
        <div class="stat-item">
            <span class="stat-label">Current Zoom:</span>
            <span class="stat-value" id="zoom-level">1.0x</span>
        </div>
    </div>
    
    <div class="legend">
        <h3>Map Legend</h3>
    
        <div class="legend-item">
            <svg width="14" height="14" viewBox="0 0 24 24">
                <circle cx="12" cy="12" r="7" fill="#2c3e50" stroke="white" stroke-width="1.5"/>
            </svg>
            Subway Station
        </div>
    
        <div class="legend-item">
            <svg width="14" height="14" viewBox="0 0 24 24">
                <path d="M4 4h16v16H4z" fill="#ecf0f1" stroke="#34495e" stroke-width="1.5"/>
                <circle cx="12" cy="12" r="3" fill="#3498db"/>
            </svg>
            Hover: Station info
        </div>
    
        <div class="legend-item">
            <svg width="14" height="14" viewBox="0 0 24 24">
                <rect x="4" y="4" width="16" height="16" rx="3" fill="#ecf0f1" stroke="#2c3e50" stroke-width="1.5"/>
                <polygon points="10,8 16,12 10,16" fill="#e74c3c"/>
            </svg>
            Click: Center view
        </div>
    
        <div class="legend-item">
            <svg width="14" height="14" viewBox="0 0 24 24">
                <rect x="4" y="4" width="16" height="16" rx="3" fill="#ecf0f1" stroke="#2c3e50" stroke-width="1.5"/>
                <path d="M12 6v12M6 12h12" stroke="#f39c12" stroke-width="2"/>
            </svg>
            Drag: Pan map
        </div>
    
        <div class="legend-item">
            <svg width="14" height="14" viewBox="0 0 24 24">
                <circle cx="12" cy="12" r="8" fill="#ecf0f1" stroke="#2c3e50" stroke-width="1.5"/>
                <path d="M12 7v4M12 13v4M7 12h4M13 12h4" stroke="#27ae60" stroke-width="2"/>
            </svg>
            Scroll: Zoom
        </div>
    </div>


    
    <div class="controls">
        <h3>Zoom Controls</h3>
        <button class="zoom-btn" id="zoom-in">+</button>
        <button class="zoom-btn" id="zoom-out">-</button>
        <button class="zoom-btn" id="reset">R</button>
        
        <div class="action-buttons">
            <button class="action-btn" id="show-all">Show All</button>
            <button class="action-btn" id="hide-all">Hide All</button>
            <button class="action-btn" id="download-data">Download CSV</button>
        </div>
        
        <div class="borough-filter">
            <h3>Search Station</h3>
            <input type="text" class="search-box" id="station-search" placeholder="Type station name...">
        </div>
        
        <div class="borough-filter">
            <h3>Filter by Borough</h3>
            <label>
                <input type="checkbox" value="Manhattan" checked>
                <span class="color-box" style="background: #e74c3c;"></span>
                Manhattan <span class="borough-count" data-borough="Manhattan">(0)</span>
            </label>
            <label>
                <input type="checkbox" value="Brooklyn" checked>
                <span class="color-box" style="background: #3498db;"></span>
                Brooklyn <span class="borough-count" data-borough="Brooklyn">(0)</span>
            </label>
            <label>
                <input type="checkbox" value="Queens" checked>
                <span class="color-box" style="background: #2ecc71;"></span>
                Queens <span class="borough-count" data-borough="Queens">(0)</span>
            </label>
            <label>
                <input type="checkbox" value="Bronx" checked>
                <span class="color-box" style="background: #f39c12;"></span>
                Bronx <span class="borough-count" data-borough="Bronx">(0)</span>
            </label>
            <label>
                <input type="checkbox" value="Staten Island" checked>
                <span class="color-box" style="background: #9b59b6;"></span>
                Staten Island <span class="borough-count" data-borough="Staten Island">(0)</span>
            </label>
        </div>
    </div>

    <script>
        const width = window.innerWidth;
        const height = window.innerHeight;

        const svg = d3.select("#map")
            .append("svg")
            .attr("width", width)
            .attr("height", height);

        const g = svg.append("g");
        const tooltip = d3.select(".tooltip");

        const boroughColors = {
            'Manhattan': '#e74c3c',
            'Brooklyn': '#3498db',
            'Queens': '#2ecc71',
            'Bronx': '#f39c12',
            'Staten Island': '#9b59b6',
            'Unknown': '#95a5a6'
        };
        
        const boroughNames = {
            1: 'Manhattan',
            2: 'Bronx',
            3: 'Brooklyn',
            4: 'Queens',
            5: 'Staten Island'
        };

        let visibleBoroughs = new Set(['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island']);
        let allStations = [];

        console.log("Script loaded, boroughNames:", boroughNames);

        const projection = d3.geoMercator()
            .center([-73.935242, 40.730610])
            .scale(60000)
            .translate([width / 2, height / 2]);

        const path = d3.geoPath().projection(projection);

        const zoom = d3.zoom()
            .scaleExtent([1, 8])
            .on("zoom", (event) => {
                g.attr("transform", event.transform);
                d3.select("#zoom-level").text(event.transform.k.toFixed(1) + "x");
            });

        svg.call(zoom);

        d3.select("#zoom-in").on("click", () => {
            svg.transition().call(zoom.scaleBy, 1.5);
        });

        d3.select("#zoom-out").on("click", () => {
            svg.transition().call(zoom.scaleBy, 0.67);
        });

        d3.select("#reset").on("click", () => {
            svg.transition().call(zoom.transform, d3.zoomIdentity);
        });

        function updateVisibleCount() {
            const count = allStations.filter(d => visibleBoroughs.has(d.properties.borough)).length;
            d3.select("#visible-stations").text(count);
        }

        Promise.all([
            d3.json("nyc_boroughs.geojson"),
            d3.json("subway_stations.geojson")
        ]).then(([boroughs, subways]) => {
            
            console.log("Data loaded!");
            console.log("Boroughs data:", boroughs);
            console.log("Number of borough features:", boroughs.features.length);
            
            allStations = subways.features;
            
            d3.select("#total-stations").text(allStations.length);
            updateVisibleCount();
            
            const boroughCounts = d3.rollup(allStations, v => v.length, d => d.properties.borough);
            boroughCounts.forEach((count, borough) => {
                d3.select('.borough-count[data-borough="' + borough + '"]').text('(' + count + ')');
            });
            
            // Draw borough polygons with colors
            g.append("g")
                .selectAll("path")
                .data(boroughs.features)
                .join("path")
                .attr("class", d => "borough borough-" + d.id)
                .attr("d", path)
                .style("filter", "drop-shadow(0px 2px 4px rgba(0,0,0,0.1))");

            // Draw subway stations
            g.append("g")
                .selectAll("circle")
                .data(subways.features)
                .join("circle")
                .attr("class", d => "subway-station borough-" + d.properties.borough.replace(/\\s+/g, '-'))
                .attr("cx", d => projection(d.geometry.coordinates)[0])
                .attr("cy", d => projection(d.geometry.coordinates)[1])
                .attr("r", 3)
                .attr("fill", d => boroughColors[d.properties.borough] || boroughColors['Unknown'])
                .on("mouseover", (event, d) => {
                    d3.select(event.target).attr("r", 6);
                    tooltip
                        .style("display", "block")
                        .html(
                            "<strong>" + d.properties.name + "</strong><br>" +
                            "Borough: " + d.properties.borough + "<br>" +
                            "Station ID: " + d.properties.stop_id
                        )
                        .style("left", (event.pageX + 10) + "px")
                        .style("top", (event.pageY + 10) + "px");
                })
                .on("mouseout", (event) => {
                    d3.select(event.target).attr("r", 3);
                    tooltip.style("display", "none");
                })
                .on("click", (event, d) => {
                    const coords = projection(d.geometry.coordinates);
                    svg.transition()
                        .duration(750)
                        .call(
                            zoom.transform,
                            d3.zoomIdentity
                                .translate(width / 2 - coords[0] * 2, height / 2 - coords[1] * 2)
                                .scale(2)
                        );
                });

            // Add borough labels at centroids (LAST so they appear on top)
            console.log("About to draw borough labels...");
            console.log("Borough features:", boroughs.features);
            
            const labelGroup = g.append("g").attr("class", "label-group");
            
            boroughs.features.forEach((feature, index) => {
                console.log(`Feature ${index}:`, feature);
                console.log(`  - id:`, feature.id);
                console.log(`  - centroid:`, path.centroid(feature));
                console.log(`  - name:`, boroughNames[feature.id]);
            });
            
            labelGroup
                .selectAll("text")
                .data(boroughs.features)
                .join("text")
                .attr("class", "borough-label")
                .attr("transform", d => {
                    const centroid = path.centroid(d);
                    return "translate(" + centroid + ")";
                })
                .attr("dy", "0.35em")
                .text(d => d.properties.BoroName);

            console.log("Borough labels should be drawn now!");

            d3.selectAll('.borough-filter input[type="checkbox"]').on('change', function() {
                const borough = this.value;
                const isChecked = this.checked;
                
                if (isChecked) {
                    visibleBoroughs.add(borough);
                } else {
                    visibleBoroughs.delete(borough);
                }
                
                const className = ".borough-" + borough.replace(/\\s+/g, '-');
                d3.selectAll(className).style("display", isChecked ? "block" : "none");
                
                updateVisibleCount();
            });

            d3.select("#show-all").on("click", () => {
                d3.selectAll('.borough-filter input[type="checkbox"]').property("checked", true).dispatch("change");
            });

            d3.select("#hide-all").on("click", () => {
                d3.selectAll('.borough-filter input[type="checkbox"]').property("checked", false).dispatch("change");
            });

            d3.select("#station-search").on("input", function() {
                const searchTerm = this.value.toLowerCase();
                
                d3.selectAll(".subway-station").classed("highlight", false);
                
                if (searchTerm.length > 0) {
                    const matches = allStations.filter(d => 
                        d.properties.name.toLowerCase().includes(searchTerm)
                    );
                    
                    matches.forEach(d => {
                        d3.selectAll(".subway-station")
                            .filter(station => station.properties.stop_id === d.properties.stop_id)
                            .classed("highlight", true);
                    });
                    
                    if (matches.length > 0) {
                        const firstMatch = matches[0];
                        const coords = projection(firstMatch.geometry.coordinates);
                        svg.transition()
                            .duration(750)
                            .call(
                                zoom.transform,
                                d3.zoomIdentity
                                    .translate(width / 2 - coords[0] * 2, height / 2 - coords[1] * 2)
                                    .scale(2)
                            );
                    }
                }
            });

            d3.select("#download-data").on("click", () => {
                const csv = [
                    ["Station Name", "Borough", "Stop ID", "Latitude", "Longitude"],
                    ...allStations.map(d => [
                        '"' + d.properties.name + '"',
                        d.properties.borough,
                        d.properties.stop_id,
                        d.geometry.coordinates[1],
                        d.geometry.coordinates[0]
                    ])
                ].map(row => row.join(",")).join("\\n");
                
                const blob = new Blob([csv], { type: "text/csv" });
                const url = URL.createObjectURL(blob);
                const a = document.createElement("a");
                a.href = url;
                a.download = "nyc_subway_stations.csv";
                a.click();
            });
        }).catch(error => {
            console.error("Error loading data:", error);
        });
    </script>
</body>
</html>'''

html_path = os.path.join(processed_root_folder, "nyc_subway_map.html") 
with open(html_path, 'w', encoding='utf-8') as f: f.write(html_content)
print(f"Updated: {html_path}")

Updated: C:\Users\Wonbin\PycharmProjects\NYC-Affordability-Transit-Access\data\processed\New York City Sales Data\nyc_subway_map.html


### Open HTML Web

In [ ]:
import http.server
import socketserver
import webbrowser
import os

# Change to the processed folder
os.chdir(processed_root_folder)

PORT = 8000

Handler = http.server.SimpleHTTPRequestHandler

with socketserver.TCPServer(("", PORT), Handler) as httpd:
    print(f"Server running at http://localhost:{PORT}/")
    print(f"Open: http://localhost:{PORT}/nyc_subway_map.html")
    print("Press Ctrl+C to stop server")
    
    # Auto-open browser
    webbrowser.open(f"http://localhost:{PORT}/nyc_subway_map.html")
    
    httpd.serve_forever()

Server running at http://localhost:8000/
Open: http://localhost:8000/nyc_subway_map.html
Press Ctrl+C to stop server


127.0.0.1 - - [13/Nov/2025 12:24:28] "GET /nyc_subway_map.html HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2025 12:24:28] "GET /subway_stations.geojson HTTP/1.1" 304 -


------

In [2]:
# import os
# import pandas as pd
# import numpy as np
# from scipy.spatial import cKDTree

# # Define core paths
# project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
# data_root = os.path.join(project_root, "data")
# interim_root = os.path.join(data_root, "interim", "New York City Sales Data")
# external_root = os.path.join(data_root, "external")
# processed_root = os.path.join(data_root, "processed")

# def compute_nearest_subway_distances():
#     """Compute distance (in meters) from each property to its nearest subway station."""

#     # Load subway stops
#     stops_path = os.path.join(project_root, "data", "external", "gtfs", "stops.txt")
#     stops_df = pd.read_csv(stops_path)
#     stops_df = stops_df.dropna(subset=["stop_lat", "stop_lon"])
#     subway_coords = np.array(list(zip(stops_df["stop_lat"], stops_df["stop_lon"])))

#     # Build KDTree for efficient spatial lookup
#     subway_tree = cKDTree(subway_coords)
#     print(f"Loaded {len(stops_df)} subway stops.")

#     # Prepare output folder
#     os.makedirs(processed_root, exist_ok=True)

#     # Loop over yearly folders
#     for year in os.listdir(interim_root):
#         year_folder = os.path.join(interim_root, year)
#         if not os.path.isdir(year_folder):
#             continue

#         print(f"\n=== Processing {year} ===")

#         for file in os.listdir(year_folder):
#             if not file.lower().endswith((".xls", ".xlsx")) or file.startswith("~"):
#                 continue

#             file_path = os.path.join(year_folder, file)
#             df = pd.read_excel(file_path)

#             if not {"LAT", "LON"}.issubset(df.columns):
#                 print(f"Skipping {file} (missing LAT/LON).")
#                 continue

#             property_coords = np.array(list(zip(df["LAT"], df["LON"])))

#             # Query nearest subway for each property
#             dist_deg, idx = subway_tree.query(property_coords, k=1)
#             df["nearest_subway_dist_m"] = dist_deg * 111_000  # convert degrees → meters

#             # Add subway stop name (optional)
#             df["nearest_subway_name"] = stops_df.iloc[idx]["stop_name"].values

#             # Save to processed folder
#             out_folder = os.path.join(processed_root, year)
#             os.makedirs(out_folder, exist_ok=True)
#             out_path = os.path.join(out_folder, f"{os.path.splitext(file)[0]}_with_subway.xlsx")

#             df.to_excel(out_path, index=False)
#             print(f"✅ Saved {out_path} ({len(df)} rows)")

#     print("\nAll yearly files updated with nearest subway distance!")

In [3]:
# compute_nearest_subway_distances()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Wonbin\\PycharmProjects\\NYC-Affordability-Transit-Access\\data\\external\\gtfs\\stops.txt'